Late updated: July 19 2022

### [Wang Materials Group @ UT Austin](https://wangmaterialsgroup.com)

# Overview

This notebook contains an example workflow for manipulating crystal structures to be used in electronic structure codes such as [Quantum ESPRESSO](https://www.quantum-espresso.org/) and [VASP](https://www.vasp.at/) using open-source materials informatics packages.

This tutorial barely scatches the surface for the capabilities of the ASE and pymatgen packages.
For example, ASE may be used to instantiate calculators and control external electronic structure codes. 

Here, we will focus on the basic task of structure manipulation and some common tasks in the research for the group.


Covered in this notebook are examples for:
- importing structures
- making supercells structures
- introducing point defects
- cleaving at a particular surface facet
- introducing adsorbates


NOTE: for instructive purposes, each cell contains the entirety of all the imports needed for the cell to run; in reality, you only need to import a package (or part of a package once)

This tutorial assumes you are familiar with the following concepts in Python:
- arrays and basic linear algebra operations
- functions
- classes, attributes. inheritence 

and the following concepts:
- basic crystallography, i.e., lattice vectors, fractional coordinates v cartesian coordinates
- Miller indices for surfaces


If any of these are unfamiliar to you, you are highly encouraged to read about them in any number of online Python tutorials, e.g., [object oriented programming](https://realpython.com/python3-object-oriented-programming/), [data types](https://realpython.com/python-data-types/), [variable types](https://www.tutorialspoint.com/python/python_variable_types.htm)

## Introduction

In any electronic structure calculation, you need to:
- know what calculation parameters to run (e.g., pseudopotential, functional, convergence parameters, etc.)
- a reasonable starting structure for the calculation to start with

We will be going over the second component- example strategies for generating and manipulating structures.

Once again, this is an example workflow to help get you get started on using helpful tools for your research. 


## Install relevant packages

Open-source Python packages for setting up and analyzing atomic materials simulations
- [pymatgen](https://pymatgen.org/) (see also [link](https://pymatgen.org/installation.html))
- [Atomic Simulation Environment (ASE)](https://wiki.fysik.dtu.dk/ase/about.html) (see also [link](https://wiki.fysik.dtu.dk/ase/install.html#))

```bash
conda install --channel conda-forge pymatgen
conda install -c conda-forge ase 
```

Crystal structure visualizers:
- [VESTA](https://jp-minerals.org/vesta/en/download.html)
- [XCRYSDEN](http://www.xcrysden.org/Download.html)

# Structure import

We will start with importing a structure file as a Python object.
Manipulation of a crystal structure is one place where object-oriented programming shines.

If object-oriented programming is a new concept to you, please read about what it is, e.g., [here](https://realpython.com/python3-object-oriented-programming/)

We shall do this using both ASE and pymatgen, which each separately define the object of a crystal structure (or molecule) differently.

We will perform some basic structure manipulation on the well-known cubic perovskite strontium titanate (SrTiO$_3$)

<img src="./files/SrTiO3.png" alt="Drawing" style="width: 200px;"/>

We will store the outputted files in a separate directory called `files`. 
You may change this or simply make a directory called `files` in your current directory.

In [ ]:
#%%bash
#mkdir files/
## uncomment this cell to make a directory from this jupyter notebook

### Some starting questions:

Using a crystal structure viewer of your choice (highlight to reveal answers)

1. What are the lattice parameter(s) of SrTiO3? <font color='white'> 3.945 Angstroms </font>
2. What is the space group of SrTiO3? <font color='white'> $Pm\bar{3}m$ </font>
3. What are the coordination numbers of Ti (light blue) and Sr (green)? <font color='white'> CN of Ti = 6, CN of Sr = 12 </font>

###  ASE

In [1]:
# Example with ASE
from ase.io import read, write

# import cif structure into ASE python object
ase_struct=read("SrTiO3.cif", format="cif")

# what is inside the structure object; compare with the cif file
print(ase_struct.cell)
print()
print(ase_struct.positions)
print(ase_struct.get_positions())
print()
print(ase_struct.get_scaled_positions())
print()
print(ase_struct[0])

[[3.94513 0.      0.     ]
 [0.      3.94513 0.     ]
 [0.      0.      3.94513]]

[[0.       0.       0.      ]
 [1.972565 1.972565 1.972565]
 [1.972565 0.       1.972565]
 [1.972565 1.972565 0.      ]
 [0.       1.972565 1.972565]]
[[0.       0.       0.      ]
 [1.972565 1.972565 1.972565]
 [1.972565 0.       1.972565]
 [1.972565 1.972565 0.      ]
 [0.       1.972565 1.972565]]

[[0.  0.  0. ]
 [0.5 0.5 0.5]
 [0.5 0.  0.5]
 [0.5 0.5 0. ]
 [0.  0.5 0.5]]

Atom('Sr', [0.0, 0.0, 0.0], tag=0, index=0)


######  Exercise: convert from cif file format to QE input file

hint: https://wiki.fysik.dtu.dk/ase/ase/io/io.html

In [2]:
## your code here
from ase.io import write

write("./files/STO.in", read("SrTiO3.cif"), format="espresso-in")

### pymatgen

In [2]:
from pymatgen.core import Lattice, Structure, Molecule

pmg_struct=Structure.from_file("SrTiO3.cif")
print(pmg_struct.lattice)
print(pmg_struct.frac_coords)

3.945130 0.000000 0.000000
0.000000 3.945130 0.000000
0.000000 0.000000 3.945130
[[0.  0.  0. ]
 [0.5 0.5 0.5]
 [0.5 0.  0.5]
 [0.5 0.5 0. ]
 [0.  0.5 0.5]]


##### Exercise: import structure file from VASP

hint: https://pymatgen.org/pymatgen.io.vasp.html

In [ ]:
## your code here

### Convert between ASE structure object and pymatgen structure object

Sometimes there are capabilities in one package that the other does not have or is perhaps not written a way that is what you are looking for. 
It is possible to translate between the structure objects of ASE and pymatgen accordingly.

In [3]:
## convert ASE structure object to pymatgen structure object
from pymatgen.io.ase import AseAtomsAdaptor
pmg_struct = AseAtomsAdaptor.get_structure(ase_struct)

print(pmg_struct.lattice)
print(pmg_struct.frac_coords)

3.945130 0.000000 0.000000
0.000000 3.945130 0.000000
0.000000 0.000000 3.945130
[[0.  0.  0. ]
 [0.5 0.5 0.5]
 [0.5 0.  0.5]
 [0.5 0.5 0. ]
 [0.  0.5 0.5]]


##### Exercise: convert pymatgen structure object to ASE structure object

hint: https://pymatgen.org/pymatgen.io.ase.html

In [ ]:
## your code here: convert pymatgen structure object to ASE structure object

# Structure Manipulation

Many of the codes we use peroidic boundary conditions, meaning the simulation cell is repeated at the cell boundary.
We frequently need to run calculations that requires some dilute concentration of something, thus necessitating what is known as a supercell. 
A supercell is simply an expansion of your unit cell translated according to the unit cell lattice vectors. 
Situations where we use a supercell:
- introducing a point defect in the dilute limit
- finite difference method for phonons
- computing spectra involving excitation of a particular atom

We will make two different sizes of supercells:
- a 3x3x3 supercell with ASE (so the unit cell has been translated 3 times in each lattice vector direction)
- a 2x2x2 supercell with pymatgen

For ASE, you should get something like the following

![](./files/SrTiO3-3x3x3.png)

Of course, for pymatgen, you will get something similar but smaller

## Make a supercell

In [4]:
# ASE: https://databases.fysik.dtu.dk/ase/_modules/ase/build/supercells.html
from ase.build.supercells import make_supercell
from ase.io import write
import numpy as np

P = 3*np.identity(3) # transformation matrix for 3x3x3 supercell
ase_supercell = make_supercell(ase_struct, P)
write("./files/SrTiO3-3x3x3.cif", ase_supercell,format="cif")

In [6]:
# pymatgen
from pymatgen.io.cif import CifWriter
pmg_struct.make_supercell([2,2,2], to_unit_cell=True)
w = CifWriter(pmg_struct)
w.write_file("./files/SrTiO3-2x2x2.cif")

## Make a point defect

Next, we'll introduce a N substitutional defect on an oxygen site.
You should get something like the following where the substitutional defect is known in dark blue:

![](./files/SrTiO3-N.png)

**Checkpoint:**
- Why do we see multiple substitutional defects in the image even though we only performed one substitution?
<font color='white'> We see the periodic images of the substitutional defect in the adjacent cell. </font>
- How far away are periodic images of the substitutional defect from each other? Is this sufficiently far to capture the dilute limit? 
<font color='white'> It depends on what we are calculating-need to perform a convergence wrt to supercell size to know for sure.</font>

In [7]:
# example with pymatgen: make a substitutional point defect
def substitute(structure,sites,species):
    """ wrapper for making substitutional defects
 
       struc (Pymatgen structure)
       sites (list of integers): sites to replace 
       species (list of strings): species of sites """
    # copy structure and leave original unmodified
    struc = structure.copy()
    
    sites = np.subtract(sites,1) # zero-indexing
    assert len(sites) == len(species)
    
    for i in range(len(sites)):
        s = sites[i]
        coord = struc.sites[s].frac_coords
        struc.replace(s,species[i],coord)
    
    return struc

pmg_defect=substitute(pmg_struct,[37+1],'N')

from pymatgen.io.cif import CifWriter
w = CifWriter(pmg_defect)
w.write_file("./files/SrTiO3-N.cif")

## Make a surface

Let's make a surface at a selected cleavage of the bulk cell.

This function depends on the concept of Miller indices, which is a notation system for crystal facets.
If Miller indices are a new concept to you, please read references in crystallography or solid-state physics, e.g., [here](https://en.wikipedia.org/wiki/Miller_index)

You should generate two slab structures.
Visualize the slab structures. 

**Checkpoint:**
- What is the main difference between the two slab structures?
<font color='white'> The termination of the surface atoms is different; one is terminated by Ti-O and the other is terminated by Sr-O </font>
- How do you know how thick the slab and vacuum spacing needs to be?
<font color='white'> It is a balance of accuracy (e.g., separating the surfaces as far from each other as possible) and computational cost. You must perform a convergence test wrt to slab thickness and vacuum thickness to really know. Relevant literature will provide a good starting point. </font>

In [ ]:
from pymatgen.core.surface import SlabGenerator,generate_all_slabs
from pymatgen.io.vasp import Poscar
from pymatgen.io.cif import CifWriter

poscar = Poscar.from_file("SrTiO3.vasp")
pmg_struct = poscar.structure

miller=[0,1,0]
slabsize=25
vacsize=15
slabgen = SlabGenerator(pmg_struct,miller,slabsize,vacsize, center_slab=True)
slabs = slabgen.get_slabs(symmetrize=True)

# output each slab in slabs
i=0
for slab in slabs:
    i+=1
    w = CifWriter(slab)
    w.write_file("./files/SrTiO3-slab"+str(i)+".cif")


## Add an adsorbate to the surface

Now, let's add an adsorbate to the surface. 
We will take the Sr-O terminated surface and add a water molecule to the surface, having effectively 100% monolayer on the surface. 

You should end up with something that looks like:

![](./files/SrTiO3-OH.png)

**Checkpoint:**

- How do we know where to add the water molecule and in what orientation?
<font color='white'> A priori we do not, but we can use a combination of physical intuition and literature search to narrow down potential candidates. For example, we know water is a polar molecule, meaning the oxygen is slightly negatively charged and hydrogens are slightly positively charged. Thus, we might expect the adsorption of water via the oxygen atom to be more energetically favorable on a metal surface site than on an oxygen surface site. </font>
- What would you need to do to make a symmetric slab with adsorbate?
<font color='white'> Add a hydroxyl group on the other side of the slab-oriented by 180$^\circ$. Make sure the slab itself is symmetric. Make sure the slab is centered - physically this does not matter, but may help with other aspects of structural manipulation. </font>
- What would you need to do to have a smaller coverage of hydroxyls?
<font color='white'> Make a supercell in the lateral directions and place adsorbate </font>
- How physical is this model?
<font color='white'> It's probably unlikely the hydroxyls all point in the vertical direction or in general are aligned. If we were to use this cell, we would only see concerted relaxation and motion of the hydroxyl groups because we have constrained the cell to only be 1x1 surface representation. We also do not know a priori that the entire Sr-O surface is covered in hydroxyls in the first place. </font>

In [ ]:
from ase.io import read
from ase.build import molecule, add_adsorbate

ase_struct=read("./files/SrTiO3-slab2.cif", format="cif")
hydroxyl = molecule('OH')

coord = ase_struct.get_positions()[0] # a surface Sr atom

# add_adsorbate(...) adds a molecule relative to max_z coordinate in the slab
# in order to add a molecule on the bottom side, we add ~ vacuum spacing of above and below the slab
add_adsorbate(ase_struct,hydroxyl,height = 18, position=(coord[0],coord[1]), mol_index=0)
write("./files/SrTiO3-OH.cif",ase_struct,format="cif")

##### Exercise: SrTiO$_3$ (010) surface at 25% coverage

Using everything you have learned so far to build a SrTiO3 (010) surface with 25% coverage of hydroxyls.
Hint: https://pymatgen.org/pymatgen.core.surface.html

Finally, if there's anything that you are not sure of --> always consult the documentation!


In [ ]:
# your code here: build a Sr-O terminated (010) surface with 25% coverage of hydroxyls

# import necessary packages or functions
from ase.build.supercells import make_supercell
from ase.build import molecule, add_adsorbate
from ase.io import write
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.core.surface import SlabGenerator
import numpy as np

## first import unit cell

## second make a supercell

## next make the supercell into a slab by introducing a vacuum spacing 

## finally, add the adsorbate to one of the slab structures